<a href="https://colab.research.google.com/github/blazingbhavneek/hyperbolic-ml/blob/main/L_Hydra%2B_Cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L-Hydra (Cleaned, with Random Graph)

## Imports

In [1]:
import numpy as np
import os, math, warnings
import sys, time, pdb, os, pickle
from numpy import *
import networkx as nx
from scipy.linalg import eigh
import sys, time, pdb, os, pickle
from scipy.optimize import minimize, fmin_l_bfgs_b
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365655 sha256=75f36a5fa55aee936ae64488aefd54fa39f0878ec6d3bbd938e629143104b186
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


## Parameters

In [73]:
nlandmarks = 101
nNodes = 250
curvature = 1
dim = 100
d = dim-1
alpha = 1
equi_adj = 0.5

# L-Hydra +
nrepeat = 8
nodes_per_run = 40
n_split = int(ceil((nNodes-nlandmarks)/float(nodes_per_run)))
maxprocs4seed = 8

## Generating Random X and D

In [70]:
def randX(N,land, d):
  n = N-land
  n_temp = n
  param = 0.4  #distance thresold
  g = nx.generators.random_geometric_graph(n_temp, param, seed = 1)
  W = np.zeros((n_temp, n_temp))
  for (x,y) in g.edges:
    W[x][y] = np.random.uniform(0.1,3)
  W_hat = W + W.T
  L = np.diag(np.matmul(W_hat,np.ones(W_hat.shape[0]))) - W_hat

  np.random.seed(1)
  X_na = np.random.multivariate_normal(np.zeros(n), np.linalg.pinv(L),d+1)

  np.random.seed(5)
  X_a = np.random.normal(0,1,size=(d+1,land))

  X = np.hstack((X_a, X_na))

  for i in range(N):
    X[0,i] = np.sqrt(1+np.linalg.norm(X[1:d+1,i])**2);

  return X


def x2hdm(N,land, X,d):
  n = N-land
  G = x2hgram(N,d,X)
  D = _arccosh(G)

  return D


def _arccosh(G):
  D = np.arccosh(-G)
  return D


def x2hgram(N,d, X):
  n = N-d

  X_ = X
  for n in range(N):
      x = X_[:,n]
      X_[:,n] = projectX(N,d,x)

  G = x2lgram(N,d, X_)

  E1 = G-valid(G)

  if np.linalg.norm(E1,'fro') > 1e-10:
      print(np.linalg.norm(E1,'fro'))
      print('inaccuracy in  x2hgram - ')

  E2 = G-l_rankPrj(G, N)

  if np.linalg.norm(E2,'fro') > 1e-10:
      print(np.linalg.norm(E2,'fro'))
      print('inaccuracy in  x2hgram -- ')
  return G


def projectX(N,d,x):
  n= N-d


  H = np.eye(d+1)
  H[0,0] = -1
  I = np.eye(d+1)

  x0 = x[0]

  eps = 1e-15
  center = 0
  error = abs(h_norm(x,N,d)+1)

  A_opt = I
  for i in range(50):
      l = 10**(-i)
      if x0 > 0:
          lambda_min = max(center-l, -1+eps)
          lambda_max = min(center+l, 1-eps)
          number = 50
      else:
          lambda_min = max(center-l*10000, 1+eps)
          lambda_max = center+l*10000
          number = 100
      lambda_list = np.linspace(lambda_min,lambda_max,num=number)
      for lambda_ in lambda_list:
          A = np.linalg.inv(I + lambda_*H)
          x_l = np.matmul(A,x)
          if abs(h_norm(x_l,N,d)+1) < error:
              A_opt = A
              error = abs(h_norm(x_l,N,d)+1)
              center = lambda_
  x_opt = np.matmul(A_opt,x)
  if error > 1e-5:
      print('error: ',error, "centre: ", center)

  return x_opt


def x2lgram(N,d,X):
  n= N-d

  H = np.eye(d+1)
  H[0,0] = -1

  G = np.matmul(np.matmul(X.T,H),X)
  return G


def h_norm(x,N,d):
  n= N-d

  H = np.eye(d+1)
  H[0,0] = -1

  x_norm = np.matmul(np.matmul(x.T,H),x)
  return x_norm


def valid(G):
  np.fill_diagonal(G, -1)
  G[G >= -1] = -1

  return G


def l_rankPrj(G, N):
  X = lgram2x(N,d,G)

  return x2lgram(N,d,X)


def lgram2x(N,d,G):
  n= N-d

  w, v = np.linalg.eig(G)
  w = w.real
  v = v.real

  lambda_0 = np.amin(w)
  ind_0 = np.argmin(w)
  w = np.delete(w, ind_0)
  ind = np.argsort(-w)
  w = -np.sort(-w)
  ind = ind[:d]
  w = w[:d]

  lambda_ = np.concatenate((abs(lambda_0), w), axis=None)
  lambda_[lambda_ <= 0] = 0
  lambda_ = np.sqrt(lambda_)

  ind_ = np.concatenate((ind_0, ind+1), axis=None)
  v = v[:, ind_]
  X = np.matmul(np.diag(lambda_),v.T)

  if X[0,0] < 0:
      X = -X

  return X

In [ ]:
#---------- Generating Random X and D ----------
d = dim-1

X = randX(nNodes,nlandmarks, d)
print("X: ",X.shape)

D = x2hdm(nNodes,nlandmarks, X,d)
D = (D+D.T)/2
print("D: ", D.shape)

L2n = D[:nlandmarks,:]
print("L2n: ", L2n.shape)
#---------- ------------------------ ----------

X:  (100, 250)
D:  (250, 250)
L2n:  (101, 250)


In [ ]:
np.save('/content/X', X)
np.save('/content/D', D)

## Objective function definitions


In [ ]:
# Frobenius Norm
def f(X,Y):
	return .5*linalg.norm(X-Y, ord='fro')**2

# Matrix Difference
def df(X,Y):
	return X-Y

# Hyperbolic Distance
def hype_dist(p1,p2,curv):
	k = curv
	c = 1./sqrt(abs(k))
	x = dot(p1,p2)
	usq1 = dot(p1,p1) + 1.0
	usq2 = dot(p2,p2) + 1.0
	udotu = x - sqrt(usq1)*sqrt(usq2)
	acoshudotu = real(arccosh(-udotu + 0j))
	d = acoshudotu * c
	return d

# Objective func for Xl: Eq 14a
def land_obj(x,**kwargs):
	dim = kwargs['dim']
	D0 = kwargs['D_landmark']
	grad = zeros(len(x))
	points = len(x[:-1])//dim
	k = x[-1]
	c = 1./sqrt(abs(k))
	P = x[:-1].reshape((points,dim))

	X = dot(P,P.T)
	Usq = diag(X) + 1.0
	U = sqrt(Usq)
	UdotU = X - outer(U,U)
	acoshUdotU = real(arccosh(-UdotU + 0j))
	fill_diagonal(acoshUdotU,0.0)

	D = acoshUdotU*c;
	D[isnan(D)] = 0
	dE = df(D,D0)
	err = f(D,D0)/2.0 # since matrix is symmetric

	# compute gradient w.r.t. k
	gradk = - dE * acoshUdotU * .5 * k * abs(k)**-2.5
	grad[-1] = sum(sum(gradk))/2.0 # divide by two since matrix is symmetric

	# compute gradient
	tol = 1e-12
	Atemp = UdotU**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./U,U)
	C = dE * A
	H1 = C * B
	hsum = sum(H1,1)
	L = len(D0)
	Ci = zeros((len(C),len(C)))
	for i in range(L):
		ui = P[i,:]
		hi = hsum[i]
		fill_diagonal(Ci,C[i])
		g = sum(dot(P.T,Ci),1) - hi*ui
		grad[dim*i:(i+1)*dim] = g

	return err, grad

# Objective func for Xn: Eq 14b
def nonland_obj(x, node, **kwargs):
	'''
	computes nonlandmark error for an entire nonlandmark
	matrix D0
	'''
	# node is index of D_nonland not D_land2nodes
	dim = kwargs['dim']
	D0 = kwargs['D_nonland']
	L = kwargs['Land_points']
	n_land, dim = L.shape
	nl_id = node
	k = kwargs['k']
	c = 1./sqrt(abs(k))

	X = dot(L,x)
	u = sqrt(dot(x,x) + 1.0)
	V = sqrt(sum(L.T**2,0) + 1.0)
	udotV = outer(u,V) - X
	d = real(arccosh(udotV + 0j)) * c
	d0 = D0[:,nl_id]
	err = f(d,d0)
	derr = df(d,d0)

	# compute gradient
	tol = 1e-12
	Atemp = udotV**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./u,V)
	C = derr * A
	H1 = C * B
	H1.shape = (n_land,1)
	C.shape = (n_land,1)

	ui = x
	Ui = tile(ui,(n_land,1)).T
	Hi = diag(H1[:,0])
	Ci = diag(C[:,0])
	G = dot(L.T,Ci) - dot(Ui,Hi)
	grad = sum(G,1)

	return err, grad

## Generating X_hat (L_Hydra )

In [ ]:
land_ids = [i for i in range(nlandmarks)]
non_land_ids = [i for i in range(nlandmarks,nNodes)]
D_land2nodes, D_land, land_ids, D_nonland, nonland_ids = L2n, L2n[:,:nlandmarks], land_ids, L2n[:,nlandmarks:], non_land_ids

In [ ]:
def hydra_landmark_fixed_curvature(curvature, dim, alpha, equi_adj, polar=False, isotropic_adj=True, hydra=False, lorentz=False):
  '''
		Parameters
		----------
		curvature : Float, optional
			Embedding curvature. The default is 1.
		dim : Int, optional
			Embedding dimension. The default is 2.
		alpha : Float, optional
			Adjusts the hyperbolic curvature. Values larger than one yield a
			more distorted embedding where points are pushed to the outer
			boundary (i.e. the ideal points) of hyperblic space. The
			interaction between code{curvature} and code{alpha} is non-linear.
			The default is 1.1.
		equi_adj : Float, optional
			Equi-angular adjustment; must be a real number between zero and
			one; only used if dim is 2. Value 0 means no ajustment, 1
			adjusts embedded data points such that their angular coordinates
			in the Poincare disc are uniformly distributed. Other values
			interpolate between the two extremes. Setting the parameter to non-
			zero values can make the embedding result look more harmoniuous in
			plots. The default is 0.5.
		polar:
			Return polar coordinates in dimension 2. (This flag is
			ignored in higher dimension).
		isotropic_adj :
			Perform isotropic adjustment, ignoring Eigenvalues
			(default: TRUE if dim is 2, FALSE else)
		hydra:
			Return radial and spherical coordinates (default: FALSE)
		lorentz:
			Return raw Lorentz coordinates (before projection to
			hyperbolic space) (default: FALSE)

		Yields
		------
		X : float array
			Hyperbolic coordinates in chosen space
    '''

    nodes = nNodes

    # sanitize/check input
    if any(np.diag(D_land) != 0):  # non-zero diagonal elements are set to zero
      np.fill_diagonal(D_land, 0)
      warnings.warn("Diagonal of input matrix D_land has been set to zero")

    if dim > len(D_land):
      raise RuntimeError(
        f"Hydra cannot embed {len(D_land)} points in {dim}-dimensions. Limit of {len(D_land)}."
      )

    if not np.allclose(D_land, np.transpose(D_land)):
      warnings.warn(
        "Input matrix D_land is not symmetric.\
        Lower triangle part is used."
      )

    if dim > 2:
      # set default values in dimension > 2
      isotropic_adj = False
      if polar:
        warnings.warn("Polar coordinates only valid in dimension two")
        polar = False
      if equi_adj != 0.0:
        warnings.warn("Equiangular adjustment only possible in dimension two.")

    # convert distance matrix to 'hyperbolic Gram matrix'
    A_land = np.cosh(np.sqrt(abs(curvature))*D_land)
    A_nonland = np.cosh(np.sqrt(abs(curvature))*D_nonland)

    nlm = len(land_ids)
    nnonlm = nodes - nlm

    # check for large/infinite values
    A_max = np.amax(A_land)
    if A_max > 1e8:
      warnings.warn(
        "Gram Matrix contains values > 1e8. Rerun with smaller\
        curvature parameter or rescaled distances."
      )
    if A_max == float("inf"):
      warnings.warn(
        "Gram matrix contains infinite values.\
        Rerun with smaller curvature parameter or rescaled distances."
      )


    # Eigendecomposition of A
    # compute leading Eigenvalue and Eigenvector
    lambda0, x0 = eigh(A_land, subset_by_index=[nlm-1, nlm-1])
    # compute lower tail of spectrum
    w, v = eigh(A_land, subset_by_index=[0,dim-1])
    print("w",len(w))
    print("v", len(v))
    idx = w.argsort()[::-1]
    # print("idx",len(idx))
    spec_tail = w[idx] # Last dim Eigenvalues
    print("")
    X_land_raw = v[:,idx] # Last dim Eigenvectors
    print("shape of  X_land_raw",  X_land_raw.shape)

    x0 = x0 * np.sqrt(lambda0) # scale by Eigenvalue
    if x0[0]<0:
      x0 = -x0 # Flip sign if first element negative

    # no isotropic adjustment: rescale Eigenvectors by Eigenvalues
    if not isotropic_adj:
      if np.array([spec_tail > 0]).any():
        warnings.warn(
          "Spectral Values have been truncated to zero. Try to use\
          lower embedding dimension"
        )
        spec_tail[spec_tail > 0] = 0
      X_land_raw = np.matmul(X_land_raw, np.diag(np.sqrt(np.maximum(-spec_tail,0))))

    X_nonland = np.matmul(np.transpose(A_nonland), np.c_[(x0/lambda0), -(X_land_raw/abs(spec_tail))])

    X_raw = np.zeros((nodes, dim))

    print(len(land_ids))
    X_raw[land_ids,:] = X_land_raw

    X_raw[nonland_ids,:] = X_nonland[:,1:(dim+1)]

    x0_full = np.zeros((nodes, 1))
    x0_full[land_ids,0] = x0[:,0]
    x0_full[nonland_ids,0] = X_nonland[:,0]
    x_min = x0_full.min()


    if hydra:
      # Calculate radial coordinate
      s = np.sqrt(np.sum(X_raw ** 2, axis=1))
      directional = X_raw / s[:, None]  # convert to directional coordinates
      r = np.sqrt((alpha*x0_full - x_min)/(alpha*x0_full + x_min)) ## multiplicative adjustment (scaling)
      X = self.poincare_to_hyper(r=r, directional=directional)
    else:
      X = X_raw

    # Calculate polar coordinates if dimension is 2
    if dim == 2:
      # calculate polar angle
      theta = np.arctan2(X[:, 0], -X[:, 1])

      # Equiangular adjustment
      if equi_adj > 0.0:
        angles = [(2 * x / nodes - 1) * math.pi for x in range(0, nodes)]
        theta_equi = np.array(
          [x for _, x in sorted(zip(theta, angles))]
        )  # Equi-spaced angles
        # convex combination of original and equi-spaced angles
        theta = (1 - equi_adj) * theta + equi_adj * theta_equi
        # update directional coordinate
        directional = np.array([np.cos(theta), np.sin(theta)]).transpose()

    if lorentz:
      X_lorentz = np.concatenate((x0_full, X), axis=1)

    return X

In [ ]:
X_hat = hydra_landmark_fixed_curvature(curvature, dim, alpha, equi_adj)

w 100
v 101

shape of  X_land_raw (101, 100)
101


<ipython-input-11-446413e345e6>:28: UserWarning: Equiangular adjustment only possible in dimension two.
  warnings.warn("Equiangular adjustment only possible in dimension two.")
<ipython-input-11-446413e345e6>:72: UserWarning: Spectral Values have been truncated to zero. Try to use          lower embedding dimension
  warnings.warn(
<ipython-input-11-446413e345e6>:79: RuntimeWarning: invalid value encountered in true_divide
  X_nonland = np.matmul(np.transpose(A_nonland), np.c_[(x0/lambda0), -(X_land_raw/abs(spec_tail))])


### Tranposing X_hat for x2hdm, to get D_hat

In [ ]:
X_hat.shape

(250, 100)

In [ ]:
X_hat = X_hat.T

In [ ]:
X_hat.shape

(100, 250)

In [ ]:
np.save('/content/X_hat', X_hat)

In [ ]:
D_hat = x2hdm(nNodes,nlandmarks, X_hat,d)

error:  1.0000000000000058 centre:  10838.383838383837
error:  1.0000000000000053 centre:  10838.39393939394
error:  1.0000000000000067 centre:  10858.585858585859
error:  1.000000000000007 centre:  10858.585858585859
error:  1.000000000000007 centre:  10878.787878787878
error:  1.000000000000006 centre:  10757.585858585859
error:  1.000000000000007 centre:  10757.585858585859
error:  1.0000000000000067 centre:  10818.191919191919
error:  1.0000000000000058 centre:  10858.585858585859
error:  1.000000000000007 centre:  10979.797979797979
error:  1.0000000000000089 centre:  10979.797979797979
error:  1.0000000000000056 centre:  10777.787878787878
error:  1.0000000000000064 centre:  10919.191919191919
error:  1.0000000000000075 centre:  11000.0
error:  1.0000000000000067 centre:  10898.9898989899
error:  1.0000000000000087 centre:  10959.59595959596
error:  1.0000000000000062 centre:  10737.383838383837
error:  1.0000000000000078 centre:  10919.191919191919
error:  1.0000000000000082 cen

LinAlgError: ignored

### (wrong) Problem with ProjectX, we don't need it anymore, need to redefine x2hdm

In [ ]:
def x2hdm_v2(N,land, X,d):
  n = N-land
  G = x2hgram_v2(N,d,X)
  D = _arccosh(G)

  return D


def x2hgram_v2(N,d, X):
  n = N-d

  G = x2lgram(N,d, X)
  return G

In [ ]:
D_hat = x2hdm_v2(nNodes,nlandmarks, X_hat,d)

<ipython-input-21-cd0d0a9876df>:35: RuntimeWarning: invalid value encountered in arccosh
  D = np.arccosh(-G)


In [ ]:
D_hat.shape

(250, 250)

In [ ]:
np.save('/content/D_hat', D_hat)

## Generating X_hat (L-Hydra +)

### Embedding Landmarks

In [12]:
X = np.load('/content/X.npy')
D = np.load('/content/D.npy')
L2n = D[:nlandmarks,:]
print("L2n: ", L2n.shape)

L2n:  (101, 250)


In [13]:
land_ids = [i for i in range(nlandmarks)]
non_land_ids = [i for i in range(nlandmarks,nNodes)]
D_land2nodes, D_land, land_ids, D_nonland, nonland_ids = L2n, L2n[:,:nlandmarks], land_ids, L2n[:,nlandmarks:], non_land_ids

In [5]:
def embed_prelandmarks(xstart,factr,param_list):
	def f(x):
		return land_obj(x,**param_list)
	bnds = [ (None, None) for i in range(len(xstart))]
	x,f,d = fmin_l_bfgs_b(f,xstart, bounds=bnds, factr=factr)
	return x,f,d

def embed_pre_nonlandmarks(xstart,node,param_list2):
	def g(x):
		return nonland_obj(x,node,**param_list2)
	bnds = [ (None, None) for i in range(len(xstart))]
	x,f,d = fmin_l_bfgs_b(g,xstart, bounds=bnds, factr=1e12)
	return x,f,d

In [6]:
def f(X,Y):
	return .5*linalg.norm(X-Y, ord='fro')**2

def df(X,Y):
	return X-Y

In [7]:
def land_obj(x,**kwargs):
	dim = kwargs['dim']
	D0 = kwargs['D_landmark']
	grad = zeros(len(x))
	points = len(x[:-1])//dim
	k = x[-1]
	c = 1./sqrt(abs(k))
	P = x[:-1].reshape((points,dim))

	X = dot(P,P.T)
	Usq = diag(X) + 1.0
	U = sqrt(Usq)
	UdotU = X - outer(U,U)
	acoshUdotU = real(arccosh(-UdotU + 0j))
	fill_diagonal(acoshUdotU,0.0)

	D = acoshUdotU*c;
	D[isnan(D)] = 0
	dE = df(D,D0)
	err = f(D,D0)/2.0 # since matrix is symmetric

	# compute gradient w.r.t. k
	gradk = - dE * acoshUdotU * .5 * k * abs(k)**-2.5
	grad[-1] = sum(sum(gradk))/2.0 # divide by two since matrix is symmetric

	# compute gradient
	tol = 1e-12
	Atemp = UdotU**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./U,U)
	C = dE * A
	H1 = C * B
	hsum = sum(H1,1)
	L = len(D0)
	Ci = zeros((len(C),len(C)))
	for i in range(L):
		ui = P[i,:]
		hi = hsum[i]
		fill_diagonal(Ci,C[i])
		g = sum(dot(P.T,Ci),1) - hi*ui
		grad[dim*i:(i+1)*dim] = g

	return err, grad

def pre_embedding(dim, D_land, landmarks0, xstart, seed=133):
	D0 = D_land
	x0 = xstart
	nL = len(D0)
	rn = random.RandomState(seed)
	if nL <= 32:
		x = append(rn.rand(1), x0.flatten())
		param_list = {'dim': dim, 'D_landmark' : D_land}
		x1,_,_ = embed_prelandmarks(x,1e10,param_list)
		return x1
	else:
		# find nearest power of two and embed
		exp = floor(log2(nL))
		if abs(exp - log2(nL)) == 0:
			# if nL is a perfect power of 2, then use lowest exp
			rL = int(2**(exp-1))
		else:
			rL = int(2**(exp))
		land_ids_new = array(sort(rn.permutation(nL)[:rL])).astype('int')
		D_land2nodes_new = D0[land_ids_new,:]
		D_land_new = D_land2nodes_new[:,land_ids_new]
		xstart_new = x0[land_ids_new]
		# embed new set of sub-landmarks
		param_list_new = {'dim': dim, 'D_landmark' : D_land_new, 'x_start0' : xstart_new}
		x_land_soln = pre_embedding(dim, D_land_new,landmarks0,xstart_new)
		# solve exactly (comment out for faster runs)
		if rL > 32 and rL <= int(landmarks0/2):
			x1temp,_,_ = embed_prelandmarks(x_land_soln,1e10,param_list_new)
			x_land_soln = x1temp
		L_new = x_land_soln[:-1].reshape((rL,dim))
		curv_new = x_land_soln[-1]
		# embed nonlandmarks
		nonland_ids_new = delete(arange(nL),land_ids_new)
		D_nonland_new = D_land2nodes_new[:,nonland_ids_new]
		nl_ids = range(len(nonland_ids_new))
		nonland_xmin = []
		for nl_id in nl_ids:
			param_list2_new = {'dim': dim, 'Land_points': L_new, 'D_nonland': D_nonland_new, 'k': curv_new}
			xstart2_0 = x0[nonland_ids_new[nl_id]]
			x2,_,_ = embed_pre_nonlandmarks(xstart2_0,nl_id,param_list2_new)
			nonland_xmin.append(x2)
		# fill in embedded points
		C0 = zeros((nL,dim))
		for ii,id in enumerate(land_ids_new):
			C0[id,:] = L_new[ii,:]
		for ii,xmin_nl in enumerate(nonland_xmin):
			C0[nonland_ids_new[ii],:] = xmin_nl
		return append(C0.flatten(),curv_new)

def nonland_obj(x, node, **kwargs):
	'''
	computes nonlandmark error for an entire nonlandmark
	matrix D0
	'''
	# node is index of D_nonland not D_land2nodes
	dim = kwargs['dim']
	D0 = kwargs['D_nonland']
	L = kwargs['Land_points']
	n_land, dim = L.shape
	nl_id = node
	k = kwargs['k']
	c = 1./sqrt(abs(k))

	X = dot(L,x)
	u = sqrt(dot(x,x) + 1.0)
	V = sqrt(sum(L.T**2,0) + 1.0)
	udotV = outer(u,V) - X
	d = real(arccosh(udotV + 0j)) * c
	d0 = D0[:,nl_id]
	err = f(d,d0)
	derr = df(d,d0)

	# compute gradient
	tol = 1e-12
	Atemp = udotV**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./u,V)
	C = derr * A
	H1 = C * B
	H1.shape = (n_land,1)
	C.shape = (n_land,1)

	ui = x
	Ui = tile(ui,(n_land,1)).T
	Hi = diag(H1[:,0])
	Ci = diag(C[:,0])
	G = dot(L.T,Ci) - dot(Ui,Hi)
	grad = sum(G,1)

	return err, grad

In [8]:
from scipy.optimize import minimize, fmin_l_bfgs_b

def pre_embed(s, param_list):
	'''
	Call pre_embedding
	'''
	dim = param_list['dim']
	D_land = param_list['D_landmark']
	landmarks0 = param_list['landmarks0']
	xstart = param_list['x_start0']
	x = pre_embedding(dim,D_land,landmarks0,xstart,seed=s)
	return land_obj(x,**param_list)[0], x

def embed_landmarks(xstart,param_list,factr=1e10):
	def f(x):
		return land_obj(x,**param_list)
	bnds = [ (None, None) for i in range(len(xstart))]
	x,f,d = fmin_l_bfgs_b(f,xstart,\
							bounds=bnds,\
							factr=factr)
	return f,x

In [9]:
def F(seed,param_list):
	'''
	Main embedding algorithm (run for multiple seeds)
	'''
	# seed = args[0]
	# param_list = args[1]
	f0,x0 = pre_embed(seed, param_list)
	return embed_landmarks(x0,param_list)


In [10]:
def run_single_landmark_embedding(param_list, nodes, land_ids, seed):
	start = time.time()

	fval, xsoln = F(seed,param_list)
	land_error = fval
	dim = param_list['dim']
	D_land = param_list['D_landmark']
	landmarks = len(D_land)
	Xsoln = xsoln[:-1].reshape(landmarks,dim)
	csoln = xsoln[-1]
	landmark_rel_error = sqrt(2*land_error)/linalg.norm(D_land,ord='fro')
	hypy_land_time = time.time() - start

	return hypy_land_time, landmark_rel_error, Xsoln, csoln

In [19]:
import sys, time, pdb, os

from numpy import *
from scipy.optimize import minimize, fmin_l_bfgs_b
from mpi4py import MPI # must use mpirun-openmpi-clang38

'''
Parallel landmark embedding
'''

# MPI init
comm = MPI.COMM_WORLD
rank = MPI.COMM_WORLD.Get_rank()
size = MPI.COMM_WORLD.Get_size()
name = MPI.Get_processor_name()


if rank == 0:
	# print("Loading data...")
	# D_land, land_ids, nonland_ids = load_data_4land(nodes, landmarks, dataloc)
	x_start_tmp = np.random.random(X.T.shape)
	# print(x_start_tmp)
	x_start = x_start_tmp[land_ids,]
else:
	D_land = None
	land_ids = None
	nonland_ids = None
	x_start = None

D_land = comm.bcast(D_land,root=0)
land_ids = comm.bcast(land_ids,root=0)
nonland_ids = comm.bcast(nonland_ids,root=0)
x_start = comm.bcast(x_start,root=0)
param_list = {'dim': dim, 'D_landmark' : D_land, 'landmarks0' : nlandmarks, 'x_start0' : x_start}

def embed_landmarks_lhp(nodes, landmarks, dim, seed):

	# START LANDMARK EMBEDDING
	# if rank == 0: print("\nStarting landmark embedding...")

	hypy_land_time, landmark_rel_error, land_coord, curv = run_single_landmark_embedding(param_list, nodes, landmarks, seed)

	return hypy_land_time, landmark_rel_error, land_coord, curv

# generate seed from rank
rn = random.RandomState(rank + 13)
seed = rn.randint(1000)

# run landmark embedding
hypy_land_time, landmark_rel_error, land_coord, curv = embed_landmarks_lhp(nNodes, nlandmarks, dim, seed)

In [20]:
print(hypy_land_time, landmark_rel_error, land_coord.shape, curv)

28.888341426849365 0.0008919196275280475 (101, 100) 0.39783459609592897


In [25]:
np.save('/content/land_coord', land_coord)
np.save('/content/seed', seed)
np.save('/content/curv', curv)

### Embedding non-landmarks

In [22]:
def pre_nonland_obj_percol(x, dim, L, k, rL_in_land_idx, d0):
	'''
	computes nonlandmark error for fixed column d0 of the
	nonlandmark matrix
	'''
	Lnew = L[rL_in_land_idx,:]
	d0 = d0[rL_in_land_idx]
	n_land, dim = Lnew.shape
	c = 1./sqrt(abs(k))

	X = dot(Lnew,x)
	u = sqrt(dot(x,x) + 1.0)
	V = sqrt(sum(Lnew.T**2,0) + 1.0)
	udotV = outer(u,V) - X
	d = real(arccosh(udotV + 0j)) * c
	err = f(d,d0)
	derr = df(d,d0)

	# compute gradient
	tol = 1e-12
	Atemp = udotV**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./u,V)
	C = derr * A
	H1 = C * B
	H1.shape = (n_land,1)
	C.shape = (n_land,1)

	ui = x
	Ui = tile(ui,(n_land,1)).T
	Hi = diag(H1[:,0])
	Ci = diag(C[:,0])
	G = dot(Lnew.T,Ci) - dot(Ui,Hi)
	grad = sum(G,1)

	return err, grad

In [55]:
def embed_nonlandmark0(xstart, dim, L, k, rL_idx, d0, factr=1e12):
    '''
    embed all nonlandmarks relative to landmarks using random start
    ignore rL_idx
    '''
    def g(x):
        return pre_nonland_obj_percol(x, dim, L, k, rL_idx, d0)

    bnds = [(None, None) for i in range(len(xstart))]
    x, f, d = fmin_l_bfgs_b(g, xstart, bounds=bnds, factr=factr)
    return f, x, d

def embed_nonlandmark_recursive(xstart, dim, L, k, rL_idxs, d0, factr=1e12):
    x = xstart.copy()
    factrs = [1e12 for r in range(len(rL_idxs))]
    factrs[-1] = 1e13

    for rr in range(len(rL_idxs)):
        f, x, d = embed_nonlandmark0(x, dim, L, k, rL_idxs[rr], d0, factr=factrs[rr])

    return f, x, d

def F2(args):
    seed = args[0]
    dim = args[1]
    L = args[2]
    k = args[3]
    rL = args[4]
    idx = args[5]
    D0 = args[6]
    node_ids = args[7] # list of nodes
    X0 = args[8]
    n_nodes = len(node_ids)
    X = []
    Fval = []
    rn2 = random.RandomState(seed)
    disp_every = .25
    niter = []
    funcalls = []
    rLs = [100 * 2 ** i for i in range(int(log2(nlandmarks / 100)) + 1)]
    # print(rLs)

    rL_idx = [sort(rn2.permutation(land_ids.copy())[:rL]) for rL in rLs]
    # print(rL_idx[0].shape)

    rL_in_land_idx = [in1d(land_ids, rL) for rL in rL_idx]
    # print(rL_in_land_idx[0].shape)
    # print(rL_in_land_idx)

    for i in range(n_nodes):
        if idx == 0:
            if i % int(n_nodes * disp_every) == 0:
                print(" %3i percent done" % (around(100 * i / float(n_nodes))))

        x0 = X0[i, :]
        ftemp, x, d = embed_nonlandmark_recursive(x0, dim, L, k, rL_in_land_idx, D0[:, node_ids[i]])
        X.append(x)
        Fval.append(ftemp)
        niter.append(d['nit'])
        funcalls.append(d['funcalls'])

    return Fval, array(X)


In [62]:
import multiprocessing as mp
import sys, time, pdb, os, argparse

from numpy import *
from scipy.optimize import minimize, fmin_l_bfgs_b
from mpi4py import MPI # must use mpirun-openmpi-clang38

'''
Parallel nonlandmark embedding
'''

# MPI init
comm = MPI.COMM_WORLD
rank = MPI.COMM_WORLD.Get_rank()
size = MPI.COMM_WORLD.Get_size()
name = MPI.Get_processor_name()

# Define static variables with argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("-n", type=int, help="number of nodes in graph (int)")
# parser.add_argument("-L", type=int, help="number of landmark nodes in graph (int)")
# parser.add_argument("-d", type=int, help="dimension of embedding (int)")
# parser.add_argument("-sp", type=int, help="choose nonlandmark split (int)")
# parser.add_argument("-s", type=str, help="source location of datafiles (str)")

# # set args to variables
# args = parser.parse_args()
# nodes = args.n
# landmarks = args.L
# dim = args.d
# nn = args.sp
# dataloc = args.s
# n_split = loadtxt(os.path.join(dataloc, 'nonland_nsplit.txt')) # total number of

nprocs2 = size

# if rank == 0:
# 	print("\n", "*"*60)
# 	print("Embedding for n = %i and L = %i in Dimension %i" %(nodes, landmarks, dim))
# 	print("*"*60)

# land_ids, nonland_ids = load_data_4nonland(nodes, landmarks, dataloc)

# if rank == 0: print("\nStarting nonlandmark embedding...")

if rank == 0:
	print("Loading data on rank = 0...")
	D0 = D_nonland
	Xstart0 = np.random.random(X.T.shape)
	L_points = land_coord
	land_err_temp_hypy = landmark_rel_error
	csoln = np.load('/content/curv.npy')
	curv = float(csoln)
else:
	D0 = None
	Xstart0 = None
	L_points = None
	land_err_temp_hypy = None
	csoln = None
	curv = None

D0 = comm.bcast(D0,root=0)
Xstart0 = comm.bcast(Xstart0,root=0)
L_points = comm.bcast(L_points,root=0)
land_err_temp_hypy = comm.bcast(land_err_temp_hypy,root=0)
csoln = comm.bcast(csoln,root=0)
curv = comm.bcast(curv,root=0)


rn = random.RandomState(131)

start = time.time()
# if rank == 0: print("starting split %i out of %i" %(nn+1,n_split))
# load split nonland matrix
_,n_nonland_temp = D0.shape

node_ids = array_split(range(n_nonland_temp),nprocs2)

# ready inputs for split
seeds = [rn.randint(1e3) for jj in range(nprocs2)]
nonland_rL = 100 # default
inputs2 = [[seeds[i],dim,L_points,curv,nonland_rL,i,D0,node_ids[i],Xstart0] for i in range(len(node_ids))]
l = [len(n) for n in node_ids]
# if rank == 0: print("Each proc gets %i nodes..." %(int(mean(l))))

# start0 = time.time()
solns2 = F2(inputs2[rank])
# end0 = time.time()

f2vals = solns2[0]
x2solns = solns2[1]
# save(os.path.join(dataloc, 'temp', 'x2s_split' + str(nn) + '_rank' + str(rank)),x2solns)
# save(os.path.join(dataloc, 'temp', 'f2s_split' + str(nn) + '_rank' + str(rank)),f2vals)

# end = time.time()
# hypy_nonland_time = end - start
# if rank == 0: print("**Parallel split took %.2f seconds" %(hypy_nonland_time))
# savetxt(os.path.join(dataloc, 'temp', 'nonland_time_split' + str(nn) + '_rank' + str(rank) + '.txt'),[hypy_nonland_time])


Loading data on rank = 0...
   0 percent done
  25 percent done
  50 percent done
  74 percent done
  99 percent done


In [63]:
x2solns.shape

(149, 100)

In [65]:
land_coord.shape

(101, 100)

In [66]:
X_hat = np.zeros((land_coord.shape[0] + x2solns.shape[0], x2solns.shape[1]))
X_hat[:land_coord.shape[0]] = land_coord
X_hat[land_coord.shape[0]:] = x2solns

In [67]:
X_hat.shape

(250, 100)

In [68]:
X_hat = X_hat.T

In [69]:
X_hat.shape

(100, 250)

In [75]:
D_hat = x2hdm(nNodes,nlandmarks, X_hat,d)

In [76]:
D_hat.shape

(250, 250)

In [77]:
np.save('/content/D_hat', D_hat)

In [83]:
def distance_mat_error(matrix1, matrix2):
    squared_diff = (matrix1 - matrix2) ** 2
    squared_diff = np.sum(squared_diff)
    error = np.sqrt(squared_diff / (matrix1.shape[0] ** 2))

    return error

In [84]:
dist_mat_error = distance_mat_error(D, D_hat)

In [85]:
dist_mat_error

2.0051824219818912